# Optimizing dynamic neural network

In [19]:
%load_ext autoreload
%autoreload 2

# external imports
import numpy as np
import pandas as pd
import seaborn as sb
import gensim
from gensim.scripts.glove2word2vec import glove2word2vec
import pylab as pl
import matplotlib.pyplot as plt
import csv
import scipy
import os.path
import sklearn as sk
import keras
from keras.layers import *
from keras.layers.core import *
import random

# internal imports
import helpers as HL
import cleaning as CL
import glove_module as GV
import neural_nets as NN



# Constants
DATA_FOLDER = "gensim_data_folder"
DATA_25DIM = DATA_FOLDER + "/gensim_glove_vectors_25dim.txt"
DATA_50DIM = DATA_FOLDER + "/gensim_glove_vectors_50dim.txt"
DATA_100DIM = DATA_FOLDER + "/gensim_glove_vectors_100dim.txt"
DATA_200DIM = DATA_FOLDER + "/gensim_glove_vectors_200dim.txt"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# TODO-list:
    - make dynamic work on exampledata
    - be able to save and load weigths
    - alter a model instead of creating new

## Experiments on our own data

In [2]:
# uses the created gensim-.txt file to create the word2vec so one can operate on it
global_vectors = GV.make_glove(DATA_200DIM)

In [3]:
#FOR TRAINING_SET
corpus_filenames = ['train_pos.txt', 'train_neg.txt','test_data.txt'] 
nr_pos_tweets = 100000
nr_neg_tweets = 100000
total_training_tweets = 200000

In [4]:
full_corpus, corpus_file_lengths = HL.create_corpus(corpus_filenames)

print("Length full corpus", len(full_corpus))
print("File lengths:", corpus_file_lengths)

Length full corpus 210000
File lengths: [100000, 100000, 10000]


In [5]:
###### Choose the corpus
processed_corpus = full_corpus

###### build vectors of all the tweets ######
num_of_dim = global_vectors.syn0.shape[1]
# seperate traindata and testdata
train_corpus = processed_corpus[:total_training_tweets:] 
predict_corpus = processed_corpus[total_training_tweets::] 
# Build a vector of all the words in a tweet
vectors = np.zeros(len(train_corpus), dtype=object)
for i, doc in enumerate(train_corpus):
    if (i % 50000) == 0:
        print("tweets processed: %.0f  of total number of tweets: %.0f" % (i,len(train_corpus)))
    vectors[i] = GV.buildWordVector(doc, num_of_dim, global_vectors)
train_document_vecs = np.concatenate(vectors)
train_document_vecs = sk.preprocessing.scale(train_document_vecs)
labels = GV.create_labels(total_training_tweets, nr_pos_tweets)
#############################################

tweets processed: 0  of total number of tweets: 200000
tweets processed: 50000  of total number of tweets: 200000
tweets processed: 100000  of total number of tweets: 200000
tweets processed: 150000  of total number of tweets: 200000


## Making a perfect neural network

In [37]:
def hyperparameter_improver(model, X, Y, init_hyper, step_sizes, epochs=3, n_folds=2, split=0.7, activation='relu',
                            epoch_threshold=2, time_sensitivity=1):
    input_dimensions = X.shape[1]
    hypers = init_hyper
    
    time_const = 0.1
    loss_const = 2
    acc_const = 0.5
    
    old_state = 999
    state_hist = []
    acc_hist = []
    hyper_hist = [] # list of tuples like [('depth', 1), ('width', 0)...] where 1 is change up, and vice versa
    
    
    curr_hyper, hyper_value = choose_hyperparameter(hypers)
    
    count = 0
    while count < 2:
        
        start = time.time()
        
        model = model(input_dimensions, hypers['width'], hypers['depth'], hypers['dropout_rate'],
                      activation=activation)
        final_model, cv_scores = GV.testing_for_dd(model, train_document_vecs, labels, epochs, n_folds, split)
        time_used = time.time() - start
        
        state, time_dominant, acc = score_state(cv_scores, time_used)
        state_hist.append(state)
        acc_hist.append(acc)
        
        act_on_move(state_hist, hypers, hyper_hist)
        
        old_state = state
        count +=1
        
    return model, hyperparameters, stepsizes

def choose_hyperparameter(hypers):
    """ choose random """
    key = random.choice(hypers.keys())
    print("is this a key:", key)
    return key
    
def choose_new_value(hypers, hyper_hist):
    """ choose either randomly either up or down, stepsize"""
    key = choose_hyperparameter(hypers)
    up_or_down = random.choice([0,1])
    
    if up_or_down:
        hypers[key] += step_sizes[key]
        hyper_hist.append((key, 1))
    else:
        hypers[key] -= step_sizes[key]
        hyper_hist.append((key, 0))
        
    return key, 

def update_value(hypers, key, up_or_down):
    if up_or_down:
        hypers[key] += step_sizes[key]
        hyper_hist.append((key, 1))
    else:
        hypers[key] -= step_sizes[key]
        hyper_hist.append((key, 0))

def act_on_move(state_hist, hypers, hyper_hist):
    """if bad move -> half decrease stepsize and go back a bit(half the distance)
    if good...really good ---> do again or 
    barely good --> do other hyperparameter"""
    last_change = state_hist[-1] > - state_hist[-2]
    # was the last move a backward move?
    #unchanged_hypers = 
    #regret_move = True if (hyper_hist[-1][0] == 
    # if the last two moves have been done on the last 
    #if (hyper_hist[-1][0] == hyper_hist[-1][0]):
        
        
    if (last_change >0.02):
        key = hyper_hist[-1][0]
        update_value(hypers, key, up_or_down)
    else:
        choose_new_value(hypers, hyper_hist)

def score_state(cv_scores, time_used, ):
    """ low score is good """
    time_punishment = time_sensitivity*time_punish_constant*time_used
    loss_punishment = np.mean(cv[:,0]) * loss_const
    acc = np.mean(cv[:,1])
    acc_punishment = acc_const/ acc
    time_dominant = round(time_punishment/(loss_punishment + acc_punishment)-0.5)
    return (loss_punishment + acc_punishment + time_punishment), time_dominant, acc   
    

In [45]:
# Neural net:
dd = NN.dynamic_dense
X = train_document_vecs
Y = labels

init_hyperparameters = {'width':200, 'depth':5, 'dropout_rate':0.2}
step_sizes = {'width':10, 'depth':1, 'dropout_rate':0.025}

model, hyperparameters, stepsizes = hyperparameter_improver(model, X, Y, init_hyperparameters, step_sizes, epochs=3, n_folds=2, split=0.7, activation='relu',
                            epoch_threshold=2, time_sensitivity=1)

TypeError: 'dict_keys' object does not support indexing

In [32]:
model, hyperparameters = hyperparameters_improver(dd, train_document_vecs, labels, init_hyperparameters)
    

NameError: name 'hyperparameters_improver' is not defined

In [9]:

input_dimensions = train_document_vecs.shape[1]
init_hyperparameters = {'width':200, 'depth':5, 'dropout_rate':0.1}
width = 300
depth = 5
epochs = 3
n_folds = 2
split = 0.7
dropout_rate=0.1

#model = NN.deep_HB(input_dimensions)
model = dd(input_dimensions, width, depth, dropout_rate=dropout_rate, activation='relu')
print(model.summary())

final_model = GV.testing_for_dd(model, train_document_vecs, labels, epochs, n_folds, split)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 300)               60300     
_________________________________________________________________
dropout_6 (Dropout)          (None, 300)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 300)               90300     
_________________________________________________________________
dropout_7 (Dropout)          (None, 300)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 300)               90300     
_________________________________________________________________
dropout_8 (Dropout)          (None, 300)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 300)               90300     
__________

In [ ]:
# classify the bitches
epochs = 5
n_folds = 2

model_scores= GV.run_k_fold([model], train_document_vecs, labels, epochs, n_folds)


In [ ]:
a = [
                  [4,5,6],
                  [7,8,9]]
np.random.shuffle(a)
a = np.array(a)
print(a.shape)

In [ ]:
print(final_model)

In [ ]:
# KAGGLE SUBMISSION
test_document_vecs = np.concatenate([GV.buildWordVector(doc, num_of_dim, global_vectors) for doc in predict_corpus])
test_document_vecs = sk.preprocessing.scale(test_document_vecs)


print("Hello world")
pred=model.predict(test_document_vecs)

pred_ones=[]
for i in pred:
    if i> 0.5:
        pred_ones.append(1)
    else:
        pred_ones.append(-1)

#CREATING SUBMISSION
ids = list(range(1,10000+1))
HL.create_csv_submission(ids, pred_ones,"testing_dd.txt")

## Prototyping